In [1]:
import os 
import json as js
import pandas as pd
import re # El paquete para tratar texto. Expresiones regulares
import time


In [2]:
import stanfordnlp

In [3]:
stfnlp=stanfordnlp.Pipeline(lang="es")

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': 'C:\\Users\\usuario\\stanfordnlp_resources\\es_ancora_models\\es_ancora_tokenizer.pt', 'lang': 'es', 'shorthand': 'es_ancora', 'mode': 'predict'}
---
Loading: mwt
With settings: 
{'model_path': 'C:\\Users\\usuario\\stanfordnlp_resources\\es_ancora_models\\es_ancora_mwt_expander.pt', 'lang': 'es', 'shorthand': 'es_ancora', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
---
Loading: pos
With settings: 
{'model_path': 'C:\\Users\\usuario\\stanfordnlp_resources\\es_ancora_models\\es_ancora_tagger.pt', 'pretrain_path': 'C:\\Users\\usuario\\stanfordnlp_resources\\es_ancora_models\\es_ancora.pretrain.pt', 'lang': 'es', 'shorthand': 'es_ancora', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': 'C:\\Users\\usuario\\stanfordnlp_resources\\es_ancora_models\\es_ancora_lemmatizer.pt', 'lang': 'es', 'shorthand': 'es_anco

In [6]:
os.chdir("C:\\Users\\usuario\\Desktop\\Universidad\\Proyectos Personales\\Proyectos-Personales")
json_files=os.listdir(os.getcwd() + "\\News Retriever\\El_Tiempo")

In [7]:
json_files

['news_El_Tiempo_2020-04-13.json',
 'news_El_Tiempo_2020-04-15.json',
 'news_El_Tiempo_2020-04-16.json',
 'news_El_Tiempo_2020-04-17.json',
 'news_El_Tiempo_2020-04-18.json',
 'news_El_Tiempo_2020-04-19.json',
 'news_El_Tiempo_2020-04-20.json',
 'news_El_Tiempo_2020-04-21.json',
 'news_El_Tiempo_2020-04-22.json',
 'news_El_Tiempo_2020-04-23.json',
 'news_El_Tiempo_2020-04-24.json',
 'news_El_Tiempo_2020-04-25.json',
 'news_El_Tiempo_2020-04-26.json',
 'news_El_Tiempo_2020-04-27.json',
 'news_El_Tiempo_2020-04-28.json',
 'news_El_Tiempo_2020-04-29.json',
 'news_El_Tiempo_2020-04-30.json',
 'news_El_Tiempo_2020-05-01.json',
 'news_El_Tiempo_2020-05-02.json',
 'news_El_Tiempo_2020-05-03.json',
 'news_El_Tiempo_2020-05-04.json',
 'news_El_Tiempo_2020-05-05.json',
 'news_El_Tiempo_2020-05-06.json',
 'news_El_Tiempo_2020-05-07.json',
 'news_El_Tiempo_2020-05-09.json',
 'news_El_Tiempo_2020-05-10.json',
 'news_El_Tiempo_2020-05-11.json',
 'news_El_Tiempo_2020-05-13.json',
 'news_El_Tiempo_202

In [8]:
def text_cleaning(df):
    
    ## REVISAR QUE ERA LO QUE ELIMINA Juan Sebastian en la clase de 11 del taller de python
    #Turn all into lowercase
    df.contenido=df.contenido.str.lower() 
    # Eliminate all the content inside a parenthesis
    df.contenido=df.contenido.str.replace(r'\([^)]*\)\"', " ")
    # Unicode character found in the documents
    df.contenido=df.contenido.str.replace("\xa0", " ")
    # Unicode character found in the documents
    df.contenido=df.contenido.str.replace("\u200b", " ")
    df.contenido=df.contenido.str.replace('"',' ')
    # Eliminate all the @__name__
    df.contenido=df.contenido.str.replace("@[a-z]+", " ")
    #Turn all possible references of Coronavirus into one
    df.contenido=df.contenido.str.replace("(covid|corona virus)", "coronavirus")
    # Eiminate punctuation signs
    df.contenido=df.contenido.str.replace('[,\.%!?\-!?\n\)\(\r]', ' ') 
    # Eliminate numbers
    df.contenido=df.contenido.str.replace('[0-9]', ' ') # Quito números
    # Eliminate the name of the newspaper from the documents
    df.contenido=df.contenido.str.replace('eltiempo', '')
    # Eliminate the excess of whitespaces
    df.contenido=df.contenido.str.replace('  +', ' ')
    return df

In [9]:
def text_lemmatizer(df):   
    df["process_time"]=0
    for i in range(0,len(df["contenido"])):
        start=time.time()
        print(i)
        num_palabras=len(df.loc[i,"contenido"].split(" "))
        if  num_palabras < 4000:
            doc= stfnlp(str(df.loc[i,"contenido"]))
            df.loc[i,"contenido"]=" ".join([word.lemma for sentence in doc.sentences for word in sentence.words if word.lemma is not None])
            end=time.time()
            df.loc[i,"process_time"]=end-start
        else:
            palabras=df.loc[i, "contenido"].split(" ")
            max_words=3300
            iteraciones= len(palabras)/max_words
            lemmatized_doc=" "
            for j in range(0,int(iteraciones)+1):
                last_j=min((j+1)*max_words, len(palabras))
                first_j=max(0,(j*max_words))
                content=" ".join(palabras[first_j:last_j])
                doc= stfnlp(content)
                lemmatized_doc=" ".join([word.lemma for sentence in doc.sentences for word in sentence.words if word.lemma is not None])
            df.loc[i,"contenido"]= lemmatized_doc                  
            end=time.time()
            df.loc[i,"process_time"]=end-start
    return df
    

In [10]:
def text_processer_step_1(df):
    df=df[(df["contenido"]!='') & (df["contenido"]!=' ')]
    df.reset_index(drop=True, inplace=True)
    df=text_cleaning(df)
    df=text_lemmatizer(df)
    return df

In [11]:
for file in json_files:
    if file != ".ipynb_checkpoints":
        new_file_name=re.findall("(?<=news_).+(?=.json)", file)[0]+"_lemmatized"
        v=new_file_name+".json"
        #dir options:
        #"C:\\Users\\usuario\\Desktop\\Universidad\\Proyectos Personales\\Dash App\\dash_app_deployment\\data"
        #"C:\\Users\\usuario\\Desktop\\Universidad\\Proyectos Personales\\Proyectos-Personales\\Dash App\\data"
        #"C:\\Users\\usuario\\Desktop\\Universidad\\Proyectos Personales\\Text Mining\\process_text"
        _dir_="C:\\Users\\usuario\\Desktop\\Universidad\\Proyectos Personales\\Text Mining\\process_text"
        json_lemmatized_files=os.listdir(_dir_)
        if v not in json_lemmatized_files:
            print(new_file_name)
            with open(os.getcwd() + "\\News Retriever\\El_Tiempo\\"+file, 'r') as d:
                info_dict =js.load(d)
                df=pd.DataFrame(info_dict)
            df=text_processer_step_1(df)
            with open(_dir_+"\\"+new_file_name+".json", 'w') as f:
                js.dump(df.to_dict(), f)
        else:
            print("The file "+file+" was already processed")


The file news_El_Tiempo_2020-04-13.json was already processed
The file news_El_Tiempo_2020-04-15.json was already processed
The file news_El_Tiempo_2020-04-16.json was already processed
The file news_El_Tiempo_2020-04-17.json was already processed
The file news_El_Tiempo_2020-04-18.json was already processed
The file news_El_Tiempo_2020-04-19.json was already processed
The file news_El_Tiempo_2020-04-20.json was already processed
The file news_El_Tiempo_2020-04-21.json was already processed
The file news_El_Tiempo_2020-04-22.json was already processed
The file news_El_Tiempo_2020-04-23.json was already processed
The file news_El_Tiempo_2020-04-24.json was already processed
The file news_El_Tiempo_2020-04-25.json was already processed
The file news_El_Tiempo_2020-04-26.json was already processed
The file news_El_Tiempo_2020-04-27.json was already processed
The file news_El_Tiempo_2020-04-28.json was already processed
The file news_El_Tiempo_2020-04-29.json was already processed
The file

C:\Users\usuario\Documents\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\usuario\Documents\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0


C:\Users\usuario\Documents\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
El_Tiempo_2020-07-30_lemmatized
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
El_Tiempo_2020-08-01_lemmatized
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
El_Tiempo_2020-08-02_lemmatized
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
